Vamos a hacer una seleccion de caracteristicas sobre una base de datos sobre vinos la cual fue explicada en detalle en el README.

Lo primero sera importar los datos revisar sus dimenciones y mostraremos las primeras 5 filas de datos.

In [1]:
import pandas as pd
# Lee el archivo
data = pd.read_csv("Vino_Tinto.csv")
print(data.shape)
print(data.columns)
print(data.head())

(1599, 12)
Index(['acidezFija', 'acidezVolatil', 'acidoCitrico', 'azucarResidual',
       'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'densidad',
       'pH', 'sulfatos', 'alcohol', 'calidad'],
      dtype='object')
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17

Vamos a separar el data frame en datos de entrenamiento y datos de prueba con una proporción
80/20. Trataremos de predecir los niveles de alcohol con nuestros modelos.

In [9]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

train, test = train_test_split(data, train_size = 0.8)
X = train.drop('alcohol', axis = 1)
Y = train.alcohol
model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                alcohol   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     277.1
Date:                Wed, 05 Feb 2025   Prob (F-statistic):               0.00
Time:                        22:05:20   Log-Likelihood:                -1112.1
No. Observations:                1279   AIC:                             2248.
Df Residuals:                    1267   BIC:                             2310.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                575.9530     14

Podemos observar que tenemos varios valores de interes gracias al p-value menos el dioxido de asufre que tiene un p-value muy alto.

Con esta informacion vamos a generar la metodología de selección hacia adelante e imprimir los índices o los nombres de las características seleccionadas. Para realizar este proceso, vamos a utilisar la funcion “SequentialFeatureSelector” de la librería
“mlxtend.feature_selection".

In [13]:
import mlxtend.feature_selection as fs
import sklearn.linear_model as lm

nmodel = lm.LinearRegression()

selector = fs.SequentialFeatureSelector(nmodel, forward = True, scoring = 'r2', k_features = (2,8), cv = 10)
selector.fit(X,Y)
print(selector.k_feature_names_)

('acidezFija', 'acidezVolatil', 'azucarResidual', 'dioxidoAzufreTotal', 'densidad', 'pH', 'sulfatos', 'calidad')


Despues de hacer nuestra seleccion secuencial podemos observar las caracteristicas que seran mas importantes en nuestro modelo.

Ahora vamos a entrenar nuesto modelo usando solo los valores significativos. Trataremos de predecir los resultados y le pediremos que nos estregue una R^2

In [31]:
from sklearn.metrics import r2_score

Xtest = test.drop('alcohol', axis = 1)
Ytest = test.alcohol
model2 = nmodel.fit(X[list(selector.k_feature_names_)], Y)
yhat = model2.predict(Xtest[list(selector.k_feature_names_)])
print('R^2=', r2_score(Ytest, yhat))

R^2= 0.6037758102302193


Ahora podemos ver el valor de de R^2 segun nuestras prediciones lo cual no parece bastante mal. ahora volveremos a reducir nuestro modelo pero ahora solo considerando las nuevas condiciones.

In [33]:

selector = fs.SequentialFeatureSelector(model2, forward = False, scoring = 'r2', k_features = (2,5), cv = 10)
selector.fit(X,Y)
print(selector.k_feature_names_)

('acidezFija', 'azucarResidual', 'densidad', 'pH', 'calidad')


Reducimos las canridades de parametros seleccionados y conseguimos el nuevo valor de R^2.

In [34]:
model3 = nmodel.fit(X[list(selector.k_feature_names_)], Y)
yhat = model3.predict(Xtest[list(selector.k_feature_names_)])
print('R^2=', r2_score(Ytest, yhat))

R^2= 0.5965003363358299


Conseguimos un valor menor de R^2 lo que se puede significar que nuestro modelo es peor. Los modelos tienen un poder explicativo similar, pero model2 es ligeramente superior (R^2 = 0.604 vs. 0.597). La diferencia es mínima, lo que sugiere que ambos capturan relaciones comparables con los datos, pero model2 explica un 0.7% más de la varianza. Ninguno alcanza un ajuste alto (R^2 > 0.7), por lo que podrían mejorarse.